In [1]:
# Instalação de bibliotecas necessárias

! pip3 install pandas
! pip3 install matplotlib
! pip3 install numpy
! pip3 install sklearn

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no lon

In [2]:
# Importação de bibliotecas de uso geral

import pandas as pd
import sklearn
import statistics 
import matplotlib.pyplot as plt
from decimal import Decimal
import numpy as np

In [3]:
# Leitura do conjunto de dados

data = pd.read_csv('tic-tac-toe.csv')
display(data.describe())
display(data.head())

,tl,tm,tr,ml,mm,mr,bl,bm,br,class
count,958,958,958,958,958,958,958,958,958,958
unique,3,3,3,3,3,3,3,3,3,2
top,x,x,x,x,x,x,x,x,x,positive
freq,418,378,418,378,458,378,418,378,418,626


,tl,tm,tr,ml,mm,mr,bl,bm,br,class
0,x,x,x,x,o,o,x,o,o,positive
1,x,x,x,x,o,o,o,x,o,positive
2,x,x,x,x,o,o,o,o,x,positive
3,x,x,x,x,o,o,o,b,b,positive
4,x,x,x,x,o,o,b,o,b,positive


In [7]:
y = []
for c in data['class']:
    result = 0
    if c == 'positive':
        result = 1
    y.append(result)

In [5]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(['x', 'o', 'b'])
X = data.drop(columns=['class'])
X['tl'] = encoder.transform(X['tl'])
X['tm'] = encoder.transform(X['tm'])
X['tr'] = encoder.transform(X['tr'])
X['ml'] = encoder.transform(X['ml'])
X['mm'] = encoder.transform(X['mm'])
X['mr'] = encoder.transform(X['mr'])
X['bl'] = encoder.transform(X['bl'])
X['bm'] = encoder.transform(X['bm'])
X['br'] = encoder.transform(X['br'])
X.head()

,tl,tm,tr,ml,mm,mr,bl,bm,br
0,2,2,2,2,1,1,2,1,1
1,2,2,2,2,1,1,1,2,1
2,2,2,2,2,1,1,1,1,2
3,2,2,2,2,1,1,1,0,0
4,2,2,2,2,1,1,0,1,0


In [107]:
# Teste de benchmark com os dados usando o AdaBoostClassifier da biblioteca sklearn

from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import KFold, cross_val_score

# Validação cruzada em 5-folds que será utilizada em todo o trabalho
k_fold = KFold(n_splits=5, shuffle=True, random_state=0)

clf = AdaBoostClassifier(n_estimators=100)
print('sklearn adaboost accuracy:', statistics.fmean(cross_val_score(clf, X, y, cv=k_fold, n_jobs=1)) * 100)

sklearn adaboost accuracy: 84.34173211169285


In [108]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
print('sklearn decision tree accuracy:', statistics.fmean(cross_val_score(clf, X, y, cv=k_fold, n_jobs=1)) * 100)

sklearn decision tree accuracy: 87.67833769633509


In [105]:
import numpy as np
import sys

def calc_alpha(error_rate):
    return 0.5 * np.log((1 - error_rate) / error_rate)

def calc_weight(w, alpha, misclassified):
    return w * np.exp(alpha * (1 if misclassified else -1))

# X: pandas dataframe
# y: list
# n_iter: integer value
def train_adaboost(X, y, n_iter):
    boost = []
    preditors = []
    total_rows = X.shape[0]
    initial_weight = 1/total_rows
    weights = []
    for col in X:
        col_values = []
        for value in X[col]:
            present = False
            for col_value in col_values:
                if col_value == value:
                    present = True
                    break
            if not present:
                col_values.append(value)
        for col_value in col_values:
            preditors.append({
                "col": col,
                "value": col_value,
                "misc": [],
                "alpha": 0,
                "TRUE": False,
                "FALSE": False
            })
        preditors.append({
            "col": 'rnd',
            "value": sys.maxsize,
            "misc": [],
            "alpha": 0,
            "TRUE": True,
            "FALSE": False
        })
        preditors.append({
            "col": 'rnd',
            "value": sys.maxsize,
            "misc": [],
            "alpha": 0,
            "TRUE": False,
            "FALSE": True
        })
    for i in range(total_rows):
        weights.append(initial_weight)
    y_index = 0
    for index, x in X.iterrows():
        for p in preditors:
            predicted_value = 0
            if p["TRUE"]:
                predicted_value = 1
            elif p["FALSE"]:
                predicted_value = 0
            else:
                predicted_value = 1 if x[p['col']] == p['value'] else 0
            if predicted_value != y[y_index]:
                p['misc'].append(y_index)
        y_index = y_index+1
    for i in range(n_iter):
        min_err = sys.maxsize
        min_err_preditor = {}
        for p in preditors:
            preditor_error_rate = 0
            for misc in p['misc']:
                preditor_error_rate = preditor_error_rate + weights[misc]
            if preditor_error_rate < min_err:
                min_err_preditor = p
                min_err = preditor_error_rate
        alpha = calc_alpha(min_err)
        min_err_preditor["alpha"] = alpha
        boost.append(min_err_preditor)
        weight_sum = 0
        for w_index in range(len(weights)):
            misclassified = False
            for misc in min_err_preditor['misc']:
                if misc == w_index:
                    misclassified = True
                    break
            weights[w_index] = calc_weight(weights[w_index], alpha, misclassified)
            weight_sum = weight_sum + weights[w_index]
        z = 1/weight_sum
        for i in range(len(weights)):
            weights[i] = weights[i] * z
    return boost

def predict_adaboost(boost, X):
    y = []
    for index, x in X.iterrows():
        predicted_value = 0
        for b in boost:
            prediction = 0
            alpha = b['alpha']
            if b["TRUE"]:
                prediction = 1
            elif b["FALSE"]:
                prediction = 0
            else:
                col = b['col']
                value = b['value']
                x_value = x[col]
                prediction = 1 if x_value == value else -1
            predicted_value = predicted_value + (prediction * alpha)
        y.append(1 if predicted_value > 0 else 0)
    return y

accuracy:  0.734375


In [114]:
accuracy_fold = []
for train_index, test_index in k_fold.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = np.array(y)[train_index], np.array(y)[test_index]
    clf = train_adaboost(X_train, y_train, 100)
    y_pred = predict_adaboost(clf, X_test)
    errors = 0
    for i in range(len(y_test)):
        if y_pred[i] != y_test[i]:
            errors = errors + 1
    accuracy_fold.append((len(y_test)-errors)/len(y_test))
print('implemented adaboost accuracy:', statistics.fmean(accuracy_fold))

implemented adaboost accuracy: 0.9436256004030487
